In [1]:
import torch
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch import Trainer
from library.models import ResNetModel
from library.datasets import (
    Caltech101DataModule,
    Caltech256DataModule,
)


torch.set_float32_matmul_precision("medium")

In [2]:
tb_logger = TensorBoardLogger(save_dir="logs", name="caltech101")
dataset = Caltech101DataModule()
checkpoint_name = "resnet50-caltech101-min-val-loss"
trainer = Trainer(
    max_epochs=10,
    logger=tb_logger,
    callbacks=[
        # Save the model with the lowest validation loss
        ModelCheckpoint(
            dirpath="checkpoints",
            monitor="val_loss",
            mode="min",
            save_top_k=1,
            filename=checkpoint_name,
            enable_version_counter=False,
        )
    ],
)

TRAIN = False

if TRAIN:
    model = ResNetModel(
        num_classes=101,
        architecture="resnet50",
        optim="sgd",
        optim_kwargs={
            "lr": 0.01,
            "momentum": 0.9,
            "weight_decay": 5e-4,
        },
    )
    trainer.fit(model, datamodule=dataset)

    # Test with the best model from the checkpoint
    results = trainer.test(datamodule=dataset, ckpt_path="best")
else:
    model = ResNetModel.load_from_checkpoint(f"checkpoints/{checkpoint_name}.ckpt")
    results = trainer.test(model, datamodule=dataset)

print(results)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/sentinel/.conda/envs/master-thesis/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 14/14 [00:02<00:00,  4.96it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      eval_accuracy         0.9192618131637573
        eval_loss           0.30455368757247925
        hp_metric           0.9192618131637573
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
[{'eval_loss': 0.30455368757247925, 'eval_accuracy': 0.9192618131637573, 'hp_metric': 0.9192618131637573}]


In [3]:
tb_logger = TensorBoardLogger(save_dir="logs", name="caltech256")
dataset = Caltech256DataModule()
checkpoint_name = "resnet50-caltech256-min-val-loss"
trainer = Trainer(
    max_epochs=20,
    logger=tb_logger,
    callbacks=[
        # Save the model with the lowest validation loss
        ModelCheckpoint(
            dirpath="checkpoints",
            monitor="val_loss",
            mode="min",
            save_top_k=1,
            filename=checkpoint_name,
            enable_version_counter=False,
        )
    ],
)

TRAIN = False

if TRAIN:
    model = ResNetModel(
        num_classes=258,
        architecture="resnet50",
        optim="sgd",
        optim_kwargs={
            "lr": 0.01,
            "momentum": 0.9,
            "weight_decay": 5e-4,
        },
    )
    trainer.fit(model, datamodule=dataset)

    # Test with the best model from the checkpoint
    results = trainer.test(datamodule=dataset, ckpt_path="best")
else:
    model = ResNetModel.load_from_checkpoint(f"checkpoints/{checkpoint_name}.ckpt")
    results = trainer.test(model, datamodule=dataset)

print(results)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 48/48 [00:11<00:00,  4.20it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      eval_accuracy         0.7614378929138184
        eval_loss           1.0309813022613525
        hp_metric           0.7614378929138184
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
[{'eval_loss': 1.0309813022613525, 'eval_accuracy': 0.7614378929138184, 'hp_metric': 0.7614378929138184}]


In [4]:
from library.models import ResNetModel
from library.datasets import Caltech101DataModule, Caltech256DataModule
from lightning import Trainer
import torch

# Load datasets
caltech101 = Caltech101DataModule()
caltech256 = Caltech256DataModule()

# Load models
caltech101_model = ResNetModel.load_from_checkpoint(
    "checkpoints/resnet50-caltech101-min-val-loss.ckpt"
)
caltech101_model.eval()
caltech256_model = ResNetModel.load_from_checkpoint(
    "checkpoints/resnet50-caltech256-min-val-loss.ckpt"
)
caltech256_model.eval()

ResNetModel(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
        

In [5]:
import pandas as pd
from torchvision.datasets import Caltech101, Caltech256

PREDICT = False

if PREDICT:
    # Predict foreign datasets
    trainer = Trainer(logger=False, enable_checkpointing=False)
    outputs_caltech101_model_caltech256 = trainer.predict(
        caltech101_model, datamodule=caltech256
    )
    outputs_caltech256_model_caltech101 = trainer.predict(
        caltech256_model, datamodule=caltech101
    )

    # Get predictions
    predictions_caltech101_model_caltech256 = torch.cat(
        outputs_caltech101_model_caltech256  # type: ignore
    ).argmax(dim=1)
    predictions_caltech256_model_caltech101 = torch.cat(
        outputs_caltech256_model_caltech101  # type: ignore
    ).argmax(dim=1)

    # Get targets and labels
    caltech101_target_tensor = torch.cat(
        [label for _, label in caltech101.predict_dataloader()]
    )
    caltech256_target_tensor = torch.cat(
        [label for _, label in caltech256.predict_dataloader()]
    )

    # Save dataframes
    df_caltech101 = pd.DataFrame(
        {
            "targets": caltech101_target_tensor.numpy(),
        }
    )
    df_caltech101.to_csv(
        "data/caltech101.csv",
        index=False,
    )
    df_caltech256 = pd.DataFrame(
        {
            "targets": caltech256_target_tensor.numpy(),
        }
    )
    df_caltech256.to_csv(
        "data/caltech256.csv",
        index=False,
    )
    df_caltech101_model_caltech256 = pd.DataFrame(
        {
            "predictions": predictions_caltech101_model_caltech256.numpy(),
        }
    )
    df_caltech101_model_caltech256.to_csv(
        "data/caltech101_model_caltech256.csv",
        index=False,
    )
    df_caltech256_model_caltech101 = pd.DataFrame(
        {
            "predictions": predictions_caltech256_model_caltech101.numpy(),
        }
    )
    df_caltech256_model_caltech101.to_csv(
        "data/caltech256_model_caltech101.csv",
        index=False,
    )

# Get label names
caltech101_labels = Caltech101(root="datasets/caltech101", download=False).categories
caltech256_labels = Caltech256(root="datasets/caltech256", download=False).categories

# Get targets
caltech101_targets = pd.read_csv("data/caltech101.csv")
caltech256_targets = pd.read_csv("data/caltech256.csv")
caltech101_model_caltech256 = pd.read_csv("data/caltech101_model_caltech256.csv")
caltech256_model_caltech101 = pd.read_csv("data/caltech256_model_caltech101.csv")

In [6]:
from library.taxonomy import Taxonomy

cross_domain_predictions = [
    (
        0,
        1,
        caltech101_model_caltech256["predictions"].to_numpy(),
    ),  # Caltech101 model predicting Caltech256
    (
        1,
        0,
        caltech256_model_caltech101["predictions"].to_numpy(),
    ),  # Caltech256 model predicting Caltech101
]
domain_targets = [
    (0, caltech101_targets["targets"].to_numpy()),
    (1, caltech256_targets["targets"].to_numpy()),
]
domain_labels = {
    0: caltech101_labels,
    1: caltech256_labels,
}
taxonomy = Taxonomy(
    cross_domain_predictions=cross_domain_predictions,
    domain_targets=domain_targets,
    domain_labels=domain_labels,
)

# Visualize the original taxonomy graph
net = taxonomy.visualize_graph(
    title="Caltech101 <=> Caltech256 Relationship Graph",
)
net.save_graph("output/caltech101_caltech256_relationships.html")

# Visualize the universal taxonomy graph
taxonomy.build_universal_taxonomy()
net = taxonomy.visualize_graph(
    title="Caltech101 <=> Caltech256 Universal Taxonomy Graph",
)
net.save_graph("output/caltech101_caltech256_universal_taxonomy.html")